# Chatbot and memory

This example demonstrates how to create a chatbot with conversation memory using LangChain.
- Use RunnableWithMessageHistory to maintain chat history across turns, 
- Store past messages in a local JSON file (messages.json) so it maintains memory across sessions.
- The script runs in a loop, allowing you to chat interactively with the AI while preserving context between messages until you type “exit,” “quit,” or “stop.”

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import FileChatMessageHistory
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOpenAI(model_name="gpt-5-nano")

In [5]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
    ("human", "{content}")
])

In [6]:
chain = prompt | llm

In [7]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: FileChatMessageHistory("messages.json"),
    input_messages_key="content",
    history_messages_key="messages",
)

In [ ]:
while True:
    content = input(">> ")

    if content.lower() in {"exit", "quit", "stop"}:
        print("Exiting chat...")
        break

    print(f"You: {content}")

    result = chain_with_history.invoke(
        {"content": content},
        config={"configurable": {"session_id": "default"}}
    )

    print(f"AI: {result.content}\n")

You: What is the capital of Germany?
AI: The capital of Germany is Berlin.

You: And what about USA?
AI: The capital of the United States (USA) is Washington, D.C. (District of Columbia).

Exiting chat...


# Older way

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
from dotenv import load_dotenv

load_dotenv()

In [ ]:
chat = ChatOpenAI()
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [ ]:
chain = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=memory
)

In [ ]:
while True:
    content = input(">> ")

    result = chain({"content": content})

    print(result["text"])